# 국민건강영양조사 제8기(2019-2021) 데이터 분석
> https://kjcn.or.kr/journal/view.php?number=728

## 수면건강 변수(X)가 우울증 위험군(Y)에 속할 확률에 어떤 영향을 미치는가


In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Data preprocessing

In [121]:
# Load KNHANES8 dataset
data_file = 'hn21_all.csv'
df_raw = pd.read_csv(data_file)

# 분석에 필요한 최종 변수 목록
selected_columns = [
    # 1. Y 변수 (정신건강)
    'mh_GAD_S',       # 범불안장애 선별도구 7항목 점수 합

    # 2. X 변수 (수면건강)
    'BP16_11',       # (만12세이상) 주중 잠자리에 든 시각_시간
    'BP16_13',       # (만12세이상) 주중 일어난 시각_시
    'BP16_21',       # (만12세이상) 주말 잠자리에 든 시각_시
    'BP16_23',       # (만12세이상) 주말 일어난 시각_시
    'BP17_2',        # 코골이
    'BP17_3',        # 주간 피곤
    'BP17_4',        # 수면 중 무호흡

    # 3. X 변수 (통제 변수)
    'age',           # 나이
    'sex',           # 성별
    'incm5',         # 소득 (5분위)
    'edu',           # 교육 수준
    'marri_1',       # 결혼 상태
    'mh_stress',     # 스트레스 인지율
    'sm_presnt',     # 현재 흡연 여부
    'dr_month',      # 월간 음주 빈도
    'pa_aerobic',    # 신체 활동
    'HE_BMI',        # BMI
    'DI1_dg',        # 고혈압 진단
    'DE1_dg',        # 당뇨병 진단

    # 4. ID 및 가중치 (데이터 관리에 필요)
    'ID',            # 개인 ID
    'year'           # 조사 연도
]

# 원본 DataFrame(df_raw)에서 필요한 컬럼만 선택
# KeyError 방지를 위해 실제 파일에 있는 컬럼만 선택
existing_cols = [col for col in selected_columns if col in df_raw.columns]
df = df_raw[existing_cols].copy()

print(f"선택된 컬럼 수: {len(existing_cols)}")


# --- 1) 연령 제한 ---
# 만 19세 이상 성인 데이터만 필터링
df = df[df['age'] >= 19].copy()
print(f"19세 이상 데이터 수: {len(df)}")


# --- 2) Y 변수 처리 (is_anxious 생성) ---
print("\n=== Y 변수 처리 중 ===")

# GAD-7 총점 결측치(88: 비해당(19세미만), 99: 모름/무응답)를 np.nan으로 변환
df['mh_GAD_S'] = df['mh_GAD_S'].replace([88, 99], np.nan)

# Y 변수가 결측치인 행은 분석에서 제외 (매우 중요)
df.dropna(subset=['mh_GAD_S'], inplace=True)

# Y 변수 생성: 7점 이상을 '불안 위험군(1)', 미만을 '정상군(0)'으로 분류
df['is_anxious'] = (df['mh_GAD_S'] >= 7).astype(int)
print(f"Y 변수('is_anxious') 생성 완료. (결측 제거 후 데이터 수: {len(df)})")


# --- 3) X 변수 전처리 ---
print("\n=== X 변수 전처리 중 ===")

# 3-1. 파생 변수 생성: 수면 시간 (Feature Engineering)
sleep_time_cols = ['BP16_11', 'BP16_13', 'BP16_21', 'BP16_23']
# 수면 시각 결측치(88, 99) np.nan으로 변환
df[sleep_time_cols] = df[sleep_time_cols].replace([88, 99], np.nan)

# (주중) 수면 시간 계산 (자정 문제 해결)
df['sleep_duration_wk'] = np.where(
    df['BP16_13'] >= df['BP16_11'],
    df['BP16_13'] - df['BP16_11'],    # 자정 안 넘는 경우 (예: 1시~7시)
    (24 - df['BP16_11']) + df['BP16_13'] # 자정 넘는 경우 (예: 23시~7시)
)

# (주말) 수면 시간 계산 (자정 문제 해결)
df['sleep_duration_we'] = np.where(
    df['BP16_23'] >= df['BP16_21'],
    df['BP16_23'] - df['BP16_21'],
    (24 - df['BP16_21']) + df['BP16_23']
)

# (보너스) 주중/주말 수면 시간 차이 (사회적 시차)
df['sleep_duration_diff'] = df['sleep_duration_we'] - df['sleep_duration_wk']


# 3-2. X 변수 결측치(Missing Value) 처리
# 1-digit 결측 코드(8, 9)를 사용하는 변수들
cols_clean_1 = ['BP17_2', 'BP17_3', 'BP17_4', 'incm5', 'edu', 'marri_1',
                'mh_stress', 'sm_presnt', 'pa_aerobic', 'DI1_dg', 'DE1_dg']
df[cols_clean_1] = df[cols_clean_1].replace([8, 9], np.nan)

# 2-digit 결측 코드(88, 99)를 사용하는 변수들
cols_clean_2 = ['dr_month']
df[cols_clean_2] = df[cols_clean_2].replace([88, 99], np.nan)

# 3-digit 이상 결측 코드(999 등)를 사용하는 변수들
df['HE_BMI'] = df['HE_BMI'].replace([999, 999.9], np.nan)


# 3-3. (선택적) 이진 변수 리코딩 (1=Yes, 2=No -> 1=Yes, 0=No)
# 로지스틱 회귀 등에서 해석을 용이하게 합니다 (0=No가 기준점이 됨)
cols_recode_1_2 = ['BP17_2', 'BP17_3', 'BP17_4', 'pa_aerobic', 'DI1_dg', 'DE1_dg']
df[cols_recode_1_2] = df[cols_recode_1_2].replace(2, 0) # 2(아니오)를 0으로 변경

print("X 변수 파생 및 결측치 처리 완료.")


# --- 4) 최종 모델링 데이터프레임 생성 ---
# 분석에 사용할 최종 X, Y 변수 목록
final_model_columns = [
    'is_anxious',             # Y
    'sleep_duration_wk',      # X (수면시간)
    'sleep_duration_we',      # X (수면시간)
    'sleep_duration_diff',    # X (수면시간)
    'BP17_2',                 # X (수면의 질)
    'BP17_3',                 # X (수면의 질)
    'BP17_4',                 # X (수면의 질)
    'age',                    # X (통제)
    'sex',                    # X (통제)
    'incm5',                  # X (통제)
    'edu',                    # X (통제)
    'marri_1',                # X (통제)
    'mh_stress',              # X (통제)
    'sm_presnt',              # X (통제)
    'dr_month',               # X (통제)
    'pa_aerobic',             # X (통제)
    'HE_BMI',                 # X (통제)
    'DI1_dg',                 # X (통제)
    'DE1_dg'                  # X (통제)
]

# 위 컬럼 중 하나라도 결측치가 있는 행은 모두 제거
df_model = df[final_model_columns].dropna().copy()

print(f"\n=== 최종 모델링 데이터 생성 완료 ===")
print(f"원본 데이터: {len(df_raw)} 행")
print(f"최종 분석 데이터: {len(df_model)} 행 (19세 이상, 결측치 제거)")

# 최종 데이터 확인
print("\n--- 최종 데이터 정보 (df_model.info()) ---")
df_model.info()

print("\n--- 최종 데이터 샘플 (df_model.head()) ---")
print(df_model.head())

선택된 컬럼 수: 22
19세 이상 데이터 수: 5952

=== Y 변수 처리 중 ===
Y 변수('is_anxious') 생성 완료. (결측 제거 후 데이터 수: 5615)

=== X 변수 전처리 중 ===
X 변수 파생 및 결측치 처리 완료.

=== 최종 모델링 데이터 생성 완료 ===
원본 데이터: 7090 행
최종 분석 데이터: 3916 행 (19세 이상, 결측치 제거)

--- 최종 데이터 정보 (df_model.info()) ---
<class 'pandas.core.frame.DataFrame'>
Index: 3916 entries, 0 to 7087
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   is_anxious           3916 non-null   int64  
 1   sleep_duration_wk    3916 non-null   float64
 2   sleep_duration_we    3916 non-null   float64
 3   sleep_duration_diff  3916 non-null   float64
 4   BP17_2               3916 non-null   float64
 5   BP17_3               3916 non-null   float64
 6   BP17_4               3916 non-null   float64
 7   age                  3916 non-null   float64
 8   sex                  3916 non-null   float64
 9   incm5                3916 non-null   float64
 10  edu                  3916 non-null   flo

/tmp/ipython-input-2782318300.py:3: DtypeWarning: Columns (140,380,513,660,671) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(data_file)


## 2. Logistic Regression & ODDS Ratio 분석하기
### is_anxious ~ BP17_2 + BP17_3 + BP17_4 +  ...


In [122]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [123]:
# df_model은 이전 단계에서 생성한 최종 데이터프레임입니다.

# --- 3.1: X, Y 분리 및 원-핫 인코딩 ---

# Y 변수 (종속 변수)
y = df_model['is_anxious']

# X 변수 (독립 변수)
X = df_model.drop('is_anxious', axis=1)

# X 변수 중 범주형/연속형 변수 이름 정의
# (리코딩해서 0/1이 된 변수들은 스케일링이나 인코딩이 불필요)
binary_features = ['BP17_2', 'BP17_3', 'BP17_4', 'pa_aerobic', 'DI1_dg', 'DE1_dg']
continuous_features = ['age', 'HE_BMI', 'sleep_duration_wk', 'sleep_duration_we', 'sleep_duration_diff']
categorical_features = ['sex', 'incm5', 'edu', 'marri_1', 'mh_stress', 'sm_presnt', 'dr_month']

# 범주형 변수(categorical_features)를 원-핫 인코딩 (One-Hot Encoding)
# drop_first=True : 더미 변수 함정(다중공선성)을 방지합니다. (e.g., sex_1만 남기고 sex_2는 제거)
X_processed = pd.get_dummies(X, columns=categorical_features, drop_first=True)

print("원-핫 인코딩 후 X 변수 컬럼:", X_processed.columns.tolist())


# --- 3.2: 훈련/테스트 데이터 분리 및 스케일링 ---

# 훈련 데이터와 테스트 데이터 분리 (80% / 20%)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# 연속형 변수(continuous_features) 스케일링
# 스케일러는 훈련 데이터(X_train)로만 '학습(fit)'해야 합니다. (데이터 누수 방지)
scaler = StandardScaler()

# 훈련 데이터에 맞춰 스케일링 적용
X_train[continuous_features] = scaler.fit_transform(X_train[continuous_features])
# 테스트 데이터에도 동일한 스케일러 적용
X_test[continuous_features] = scaler.transform(X_test[continuous_features])

print("\n데이터 분리 및 스케일링 완료.")


# --- 3.3: 로지스틱 회귀 모델 학습 ---

# 로지스틱 회귀 모델 생성 및 학습
# C=1.0 : 정규화(Regularization) 강도 조절 (낮을수록 강한 정규화)
# 'l2' (Ridge) : 과적합(Overfitting)을 방지하는 L2 정규화 사용
log_reg = LogisticRegression(penalty='l2', C=1.0, solver='liblinear', random_state=42, max_iter=1000, class_weight='balanced')

log_reg.fit(X_train, y_train)

# (참고) 모델 성능 평가
y_pred = log_reg.predict(X_test)
print("\n=== 모델 정확도 ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))


# --- 3.4: 결과 해석 (오즈비 계산) ---
print("\n=== 로지스틱 회귀 계수 및 오즈비(Odds Ratio) ===")

# 원-핫 인코딩된 X의 컬럼명 가져오기
feature_names = X_train.columns

# 모델의 계수(coefficients) 가져오기
coefficients = log_reg.coef_[0]

# 계수를 지수 변환(exp)하여 오즈비(Odds Ratio) 계산
odds_ratios = np.exp(coefficients)

# 해석하기 쉬운 DataFrame으로 만들기
coeff_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient (Log-Odds)': coefficients,
    'Odds Ratio': odds_ratios
})

# 오즈비 기준으로 내림차순 정렬 (영향력이 큰 변수 확인)
coeff_df = coeff_df.sort_values(by='Odds Ratio', ascending=False)

print(coeff_df)

원-핫 인코딩 후 X 변수 컬럼: ['sleep_duration_wk', 'sleep_duration_we', 'sleep_duration_diff', 'BP17_2', 'BP17_3', 'BP17_4', 'age', 'pa_aerobic', 'HE_BMI', 'DI1_dg', 'DE1_dg', 'sex_2.0', 'incm5_2.0', 'incm5_3.0', 'incm5_4.0', 'incm5_5.0', 'edu_2.0', 'edu_3.0', 'edu_4.0', 'marri_1_2.0', 'mh_stress_1.0', 'sm_presnt_1.0', 'dr_month_1.0']

데이터 분리 및 스케일링 완료.

=== 모델 정확도 ===
Accuracy: 0.7921
              precision    recall  f1-score   support

           0       0.98      0.80      0.88       735
           1       0.19      0.73      0.31        49

    accuracy                           0.79       784
   macro avg       0.59      0.77      0.59       784
weighted avg       0.93      0.79      0.84       784


=== 로지스틱 회귀 계수 및 오즈비(Odds Ratio) ===
                Feature  Coefficient (Log-Odds)  Odds Ratio
20        mh_stress_1.0                2.537812   12.651957
4                BP17_3                0.819116    2.268494
11              sex_2.0                0.568614    1.765819
19          marr

## 3. XGBoost로 중요한 특성 찾기
개별 모델의 한계를 극복하고, "정신건강에 가장 큰 영향을 미치는 요인"의 종합 순위 매기기

In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier # XGBoost 임포트
from sklearn.metrics import classification_report
from collections import Counter # 비율 계산을 위해 임포트

In [125]:
# X_train, X_test, y_train, y_test는 이전 단계에서
# 스케일링과 원-핫 인코딩이 완료된 데이터라고 가정합니다.

# --- 4.1: scale_pos_weight 계산 ---
# 훈련 데이터(y_train)의 클래스 비율을 계산합니다.
counter = Counter(y_train)
count_0 = counter[0] # 다수 클래스 (정상군)
count_1 = counter[1] # 소수 클래스 (불안 위험군)

scale_pos_weight = count_0 / count_1

print(f"=== XGBoost 파라미터 계산 ===")
print(f"훈련 데이터: 정상군(0)={count_0}명, 위험군(1)={count_1}명")
print(f"scale_pos_weight 값: {scale_pos_weight:.2f}")


# --- 4.2: XGBoost 모델 학습 ---
# use_label_encoder=False : 최신 XGBoost의 경고 메시지를 없애기 위함
# eval_metric='logloss' : 분류 문제의 표준 평가 지표
xgb_model = XGBClassifier(
    n_estimators=100,
    scale_pos_weight=scale_pos_weight, # <--- 핵심 파라미터!
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1
)

print("\n=== XGBoost 모델 학습 시작... ===")
xgb_model.fit(X_train, y_train)
print("학습 완료.")


# --- 4.3: 성능 평가 ---
y_pred_xgb = xgb_model.predict(X_test)

print("\n=== XGBoost 모델 정확도 ===")
print(classification_report(y_test, y_pred_xgb))


# --- 4.4: 특성 중요도(Feature Importances) 추출 ---

# 특성 중요도 추출
importances = xgb_model.feature_importances_

# X_train의 컬럼명 가져오기
feature_names = X_train.columns

# 컬럼명과 중요도 점수를 매칭하여 DataFrame 생성
importance_df_xgb = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# 중요도(Importance)가 높은 순서대로 정렬
importance_df_xgb = importance_df_xgb.sort_values(by='Importance', ascending=False)

print("\n=== 특성 중요도 (Top 10) ===")
print(importance_df_xgb.head(10))

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [07:18:47] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


=== XGBoost 파라미터 계산 ===
훈련 데이터: 정상군(0)=2863명, 위험군(1)=269명
scale_pos_weight 값: 10.64

=== XGBoost 모델 학습 시작... ===
학습 완료.

=== XGBoost 모델 정확도 ===
              precision    recall  f1-score   support

           0       0.94      0.93      0.94       735
           1       0.14      0.16      0.15        49

    accuracy                           0.89       784
   macro avg       0.54      0.55      0.55       784
weighted avg       0.89      0.89      0.89       784


=== 특성 중요도 (Top 10) ===
                Feature  Importance
20        mh_stress_1.0    0.360932
4                BP17_3    0.059188
18              edu_4.0    0.047801
2   sleep_duration_diff    0.037051
16              edu_2.0    0.034394
12            incm5_2.0    0.033324
17              edu_3.0    0.033066
0     sleep_duration_wk    0.030450
15            incm5_5.0    0.030344
11              sex_2.0    0.030066


In [126]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

#5.1. SMOTE 객체 생성
smote = SMOTE(random_state=42)

# 5.2. 훈련 데이터(X_train, y_train)에만 SMOTE를 적용
print("SMOTE 오버샘플링 시작...")
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print("오버샘플링 완료.")
print(f"원본 훈련 데이터: {Counter(y_train)}")
print(f"SMOTE 적용 후 훈련 데이터: {Counter(y_train_resampled)}")

# 5.3. '균형이 맞춰진' 새 데이터로 XGBoost 모델 재학습
# (SMOTE를 사용했으므로 scale_pos_weight 파라미터는 제거합니다)
xgb_model_smote = XGBClassifier(
    n_estimators=100,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1
)

print("\nSMOTE 적용 XGBoost 모델 학습 시작...")
xgb_model_smote.fit(X_train_resampled, y_train_resampled)
print("학습 완료.")

# 5.4. 최종 성능 및 특성 중요도 확인
y_pred_smote = xgb_model_smote.predict(X_test)

print("\n=== SMOTE + XGBoost 모델 정확도 ===")
print(classification_report(y_test, y_pred_smote))

# --- 5.4: 특성 중요도(Feature Importances) 추출 ---

importances = xgb_model_smote.feature_importances_
feature_names = X_train_resampled.columns

importance_df_xgb_smote = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# 중요도(Importance)가 높은 순서대로 정렬
importance_df_xgb_smote = importance_df_xgb_smote.sort_values(by='Importance', ascending=False)

print("\n=== 특성 중요도 (Top 10) ===")
print(importance_df_xgb_smote.head(10))

SMOTE 오버샘플링 시작...
오버샘플링 완료.
원본 훈련 데이터: Counter({0: 2863, 1: 269})
SMOTE 적용 후 훈련 데이터: Counter({0: 2863, 1: 2863})

SMOTE 적용 XGBoost 모델 학습 시작...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [07:18:47] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


학습 완료.

=== SMOTE + XGBoost 모델 정확도 ===
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       735
           1       0.19      0.18      0.19        49

    accuracy                           0.90       784
   macro avg       0.57      0.57      0.57       784
weighted avg       0.90      0.90      0.90       784


=== 특성 중요도 (Top 10) ===
                Feature  Importance
20        mh_stress_1.0    0.555074
11              sex_2.0    0.056627
4                BP17_3    0.048875
7            pa_aerobic    0.029747
19          marri_1_2.0    0.028802
3                BP17_2    0.027466
10               DE1_dg    0.026643
9                DI1_dg    0.021804
2   sleep_duration_diff    0.021508
21        sm_presnt_1.0    0.018657


# **수면과 정신건강의 관계 분석 최종 보고서**

**1. 분석 요약 (Executive Summary)**
본 분석은 2021년 국민건강영양조사(19세 이상) 데이터를 활용하여, 수면 건강 및 기타 생활 습관 요인이 불안장애 위험군(`is_anxious=1`)에 미치는 영향을 규명하고자 하였다. 데이터의 심각한 불균형 문제를 해결하기 위해 로지스틱 회귀(LR), XGBoost 등 다양한 모델과 `class_weight`, `scale_pos_weight`, `SMOTE` 기법을 적용하여 비교 분석하였다.

분석 결과, **`class_weight='balanced'`를 적용한 로지스틱 회귀 모델**이 불안 위험군을 **73%의 재현율(Recall)로 탐지**하며 가장 우수한 성능을 보였다. 이 모델을 통해 **스트레스가 불안 위험을 12.7배 높이는 압도적인 1위 요인**임을 확인했으며, **'주간 피로감'으로 대표되는 수면의 질 저하(2.3배)가 그 뒤를 잇는 핵심 위험 요인**임을 입증했다. 또한, 충분한 수면 시간은 불안을 완화하는 보호 요인으로 작용했다.

**2. 모델 성능 비교: 최적의 모델을 찾아서**


분석 초기, 표준적인 앙상블 모델(Random Forest, XGBoost)들은 극심한 데이터 불균형(정상군:위험군 ≈ 25:1)으로 인해 불안 위험군을 거의 예측하지 못했다(Recall 0%~18%). 이는 `scale_pos_weight`나 `SMOTE` 같은 고급 기법으로도 완전히 해결되지 않았다.

반면, **`class_weight='balanced'`를 적용한 로지스틱 회귀(LR) 모델**은 비록 정밀도(Precision)는 낮았지만, **실제 위험군 10명 중 7명 이상을 찾아내는(Recall 73%)** 실용적인 탐지 성능을 보여주어, 본 분석의 최종 해석 모델로 채택되었다.

| 모델 | **불안 위험군(1) 재현율(Recall)** | **평가** |
| :--- | :---: | :--- |
| **로지스틱 회귀 + `class_weight`** | **73%** | **성공 (최적 모델)** |
| XGBoost + `scale_pos_weight` | 16% | 실패 |
| XGBoost + SMOTE | 18% | 실패 |

**3. 주요 발견 및 해석 (From Best Model: Logistic Regression)**

#### **A. 가장 강력한 위험 요인: 스트레스**
모든 모델에서 일관되게 가장 중요한 변수로 지목된 **스트레스(`mh_stress`)**는, 최종 모델에서 불안 위험을 **12.7배**나 높이는 압도적인 요인으로 나타났다. 이는 정신건강을 논할 때 스트레스 관리가 최우선 과제임을 시사한다.

#### **B. 수면의 질과 불안의 관계: "어떻게 자는가?"**
수면의 질 저하는 불안 위험을 높이는 명백한 위험 신호였다.
* **주간 피로감 (`BP17_3`):** 낮에 피로감을 느끼는 것은 불안 위험을 **2.27배** 높였다. 이는 밤새 뒤척이거나 깊게 잠들지 못하는 등 수면의 질이 정신건강과 직결됨을 보여준다.
* **수면 중 무호흡 (`BP17_4`):** 수면 중 무호흡 증상은 불안 위험을 **1.26배** 높였다.

#### **C. 수면의 양과 불안의 관계: "얼마나 자는가?"**
충분한 수면은 불안을 완화하는 보호 요인으로 작용했다.
* **주말 수면 시간 (`sleep_duration_we`):** 주말 수면이 1시간 늘어날수록 불안 위험이 **약 8% 감소**했다. 주중에 부족했던 잠을 주말에 보충하는 것이 정신건강에 긍정적인 영향을 미칠 수 있음을 시사한다.

**4. 한계점 및 제언**
* **한계점:** 본 분석의 최적 모델은 위험군을 놓치지 않고 찾아내는 데(높은 재현율) 초점을 맞춘 결과, 정상인을 위험군으로 잘못 판단하는(낮은 정밀도, 19%) 한계가 있다.
* **제언:** 본 분석 결과는 **"스트레스 관리"**와 **"수면의 질 개선(주간 피로감 해소)"**이 국민 정신건강 증진을 위한 핵심 개입 요소임을 강력히 시사한다. 향후 정밀도를 개선하기 위한 모델 튜닝 및 추가 변수 탐색 연구가 필요하다.

---
### **최종 결론**
이번 데이터 분석 여정을 통해, 우리는 **스트레스**가 정신건강을 위협하는 가장 큰 요인이며, **수면의 질과 양** 역시 불안과 매우 밀접한 관계를 맺고 있음을 통계적으로 입증했다. 특히, 단순히 '얼마나 오래 자는가'를 넘어 '얼마나 잘 자는가'가 정신건강에 매우 중요한 요소임을 확인한 것은 본 분석의 핵심적인 성과이다.